<a href="https://colab.research.google.com/github/dnrghks5583/Way-that-set-threshold-for-setting-PMD/blob/main/CALTECH-256/Continue_ResNet_CALTECH_256.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from keras.models import load_model
import tensorflow as tf
import os

drive.mount('/content/gdrive')

#load most recent saved model
directory_name = 'CALTECH-256_no_augmentation_models_with_lr_e-3'
learning_rate_variable = int(directory_name[-1])
image_size = (128, 128)
path = '/content/gdrive/MyDrive/Colab Notebooks/ResNet_졸과/GP/pickle_256'

os.chdir(path)


directory_path = path+'/' + directory_name + '/'
file_name_and_time_lst = []

for saved_models in os.listdir(f"{directory_path}"):
    written_time = os.path.getctime(f"{directory_path}{saved_models}")
    file_name_and_time_lst.append((saved_models, written_time))
 
sorted_file_lst = sorted(file_name_and_time_lst, key=lambda x: x[1], reverse=True)
latest_model = sorted_file_lst[0][0] #originally [0][0]
print('\nLoaded model name : ' , latest_model)

#pickup from the last epoch
initial_epoch = int(latest_model[len(latest_model)-6:-3])

model = tf.keras.models.load_model(directory_path + latest_model)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).

Loaded model name :  CALTECH-256_ResNet56v1_model.085.h5


In [ ]:
"""
ResNet v1
[a] Deep Residual Learning for Image Recognition
https://arxiv.org/pdf/1512.03385.pdf
ResNet v2
[b] Identity Mappings in Deep Residual Networks
https://arxiv.org/pdf/1603.05027.pdf

https://github.com/dnrghks5583/Way-that-set-threshold-for-setting-PMD/blob/main/Caltech101/caltech101_resnet.ipynb
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input
from tensorflow.keras.layers import Flatten, add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
import numpy as np
import math

# training parameters
batch_size = 128 # orig paper trained all networks with batch_size=128
epochs = 200
num_classes = 257
data_augmentation = True

# subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 9

# model version
# orig paper: version = 1 (ResNet v1), 
# improved ResNet: version = 2 (ResNet v2)
version = 1

In [ ]:
import gzip
import pickle

with gzip.open('caltech256_x_trainData.pickle', 'rb') as f:
    x_train = pickle.load(f)
    
with gzip.open('caltech256_x_testData.pickle', 'rb') as f:
    x_test = pickle.load(f)
    
with gzip.open('caltech256_y_trainData.pickle', 'rb') as f:
    y_train = pickle.load(f)
    
with gzip.open('caltech256_y_testData.pickle', 'rb') as f:
    y_test = pickle.load(f)

"""
# score for previous trained model
scores = model.evaluate(x_test,
                        y_test,
                        batch_size=batch_size,
                        verbose=0)

print('Test loss:', scores[0])
print('Test accuracy:', scores[1])  
"""

"\n# score for previous trained model\nscores = model.evaluate(x_test,\n                        y_test,\n                        batch_size=batch_size,\n                        verbose=0)\n\nprint('Test loss:', scores[0])\nprint('Test accuracy:', scores[1])  \n"

In [ ]:
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# input image dimensions.
input_shape = x_train.shape[1:]

# normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# if subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('Model_Type : ' , model_type)
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)
# convert class vectors to binary class matrices.
#y_train = to_categorical(y_train, num_classes)
#y_test = to_categorical(y_test, num_classes)

Model_Type :  ResNet56v1
x_train shape: (24485, 128, 128, 3)
24485 train samples
6122 test samples
y_train shape: (24485, 257)


In [ ]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1*(10**(-learning_rate_variable)) #originally 3
    if epoch > 180:
        lr *= 0.5*(10**(-learning_rate_variable)) #originally 3
    elif epoch > 160:
        lr *= 1*(10**(-learning_rate_variable)) #originally 3
    elif epoch > 120:
        lr *= 1*(10**(-learning_rate_variable+1)) #originally 2
    elif epoch > 80:
        lr *= 1*(10**(-learning_rate_variable+2)) #originally 1
    print('Learning rate: ', lr)
    return lr

In [ ]:
# prepare model model saving directory.

save_dir = os.path.join(os.getcwd(), directory_name)
model_name = 'CALTECH-256_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)
                             


In [ ]:
lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

#callbacks = [lr_reducer, lr_scheduler]
callbacks = [checkpoint, lr_reducer, lr_scheduler]

# run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # this will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False)

Using real-time data augmentation.


In [ ]:
    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    steps_per_epoch =  math.ceil(len(x_train) / batch_size)
    # fit the model on the batches generated by datagen.flow().
    model.fit(x=datagen.flow(x_train, y_train, batch_size=batch_size),
              initial_epoch = initial_epoch,
              verbose=1,
              epochs=epochs,
              validation_data=(x_test, y_test),
              steps_per_epoch=steps_per_epoch,
              callbacks=callbacks)


# score trained model
scores = model.evaluate(x_test,
                        y_test,
                        batch_size=batch_size,
                        verbose=0)

print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 86/200
Learning rate:  0.0001
192/192 [==============================] - 125s 634ms/step - loss: 0.5322 - acc: 0.9261 - val_loss: 3.7734 - val_acc: 0.5124

Epoch 00086: val_acc improved from -inf to 0.51241, saving model to /content/gdrive/My Drive/Colab Notebooks/ResNet_졸과/GP/pickle_256/CALTECH-256_no_augmentation_models_with_lr_e-3/CALTECH-256_ResNet56v1_model.086.h5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 87/200
Learning rate:  0.0001
192/192 [==============================] - 121s 628ms/step - loss: 0.5218 - acc: 0.9280 - val_loss: 3.8482 - val_acc: 0.5111

Epoch 00087: val_acc did not improve from 0.51241
Epoch 88/200
Learning rate:  0.0001
192/192 [==============================] - 121s 629ms/step - loss: 0.5146 - acc: 0.9309 - val_loss: 3.9753 - val_acc: 0.5091

Epoch 00088: val_acc did not improve from 0.51241
Epoch 89/200
Learning rate:  0.0001
192/192 [==============================] - 121s 628ms/step - loss: 0.4996 - acc: 0.9355 - val_loss: 3.8986 - val_acc: 0.5126

Epoch 00089: val_acc improved from 0.51241 to 0.51258, saving model to /content/gdrive/My Drive/Colab Notebooks/ResNet_졸과/GP/pickle_256/CALTECH-256_no_augmentation_models_with_lr_e-3/CALTECH-256_ResNet56v1_model.089.h5
Epoch 90/200
Learning rate:  0.0001
192/192 [==============================] - 121s 629ms/step - loss: 0.4983 - acc: 0.9358 - val_loss: 3.8962 - val_acc: 0.5127

Epoch 00090: val_acc improved 